In [5]:
import cv2 as cv
import numpy as np
from skimage.feature import graycomatrix, graycoprops
import matplotlib.pyplot as plt

1. Problem Description:

The goal of this exercise is to train and evaluate two classifiers. The task for the classifiers is to distinguish
between different regions (classes) in the images you already know from Exercise 1 and 2.
Patches (20 x 20 pixels) of the OCT fore- and background as well as the X- Ray fore- and background were
extracted, see Figure above. The textural features you know from Exercise 2 were extracted for each
patch and are stored in the attached csv file.
Two files are attached to the exercise:

● “XL.csv”: The design matrix was constructed based on the textural features of Exercise 2 and can
be found in this file. In addition, the last column of this data contains the class labels (1 = OCT
foreground, 2 = OCT background, 3 = X-Ray background, 4 = X-Ray foreground)
● “patches.tif”: The image containing all patches.



2. Feature Selection

In [6]:
# load the csv file
%pip install pandas

import pandas as pd

# Read CSV file
df = pd.read_csv('XL.csv')

# Display the DataFrame
print(df)


Note: you may need to restart the kernel to use updated packages.
     Contrast_0  Contrast_45  Contrast_90  Contrast_135  Correlation_0  \
0      0.665789     1.036011     0.489474      0.858726       0.881278   
1      0.344737     0.590028     0.471053      0.659280       0.906595   
2      0.626316     1.138504     0.557895      0.789474       0.871879   
3      0.421053     0.836565     0.610526      0.778393       0.915127   
4      0.263158     0.612188     0.555263      0.695291       0.919149   
..          ...          ...          ...           ...            ...   
671    0.313158     0.468144     0.497368      0.681440       0.777813   
672    0.260526     0.301939     0.305263      0.434903       0.734972   
673    0.165789     0.277008     0.263158      0.307479       0.653294   
674    0.223684     0.423823     0.386842      0.451524       0.756235   
675    0.128947     0.185596     0.102632      0.135734       0.514378   

     Correlation_45  Correlation_90  Correlat

Data includes 676 examples labeled in 4 groups with 20 features (Contrast, Correlation, Energy, Homogeneitiy and Entropy for 4 different angles: 0°,45°,90°,135°). One column is a labeling column, so not really an own feature.

In [7]:
# adjust dataset

df = df.drop(columns=['Label'])
# Assuming df is your DataFrame

# Select the first and last 52 examples for the test set
test_set = pd.concat([df.head(52), df.tail(52)])

# Select the remaining examples for the training set
train_set = df.iloc[52:-52]

# Display the shapes of the resulting sets
print("Training set shape:", train_set.shape)
print("Test set shape:", test_set.shape)

print(df)

Training set shape: (572, 20)
Test set shape: (104, 20)
     Contrast_0  Contrast_45  Contrast_90  Contrast_135  Correlation_0  \
0      0.665789     1.036011     0.489474      0.858726       0.881278   
1      0.344737     0.590028     0.471053      0.659280       0.906595   
2      0.626316     1.138504     0.557895      0.789474       0.871879   
3      0.421053     0.836565     0.610526      0.778393       0.915127   
4      0.263158     0.612188     0.555263      0.695291       0.919149   
..          ...          ...          ...           ...            ...   
671    0.313158     0.468144     0.497368      0.681440       0.777813   
672    0.260526     0.301939     0.305263      0.434903       0.734972   
673    0.165789     0.277008     0.263158      0.307479       0.653294   
674    0.223684     0.423823     0.386842      0.451524       0.756235   
675    0.128947     0.185596     0.102632      0.135734       0.514378   

     Correlation_45  Correlation_90  Correlation_135  E